# 60行の `numpy` で学ぶGPT

### 入力 / 出力

```python
# def <関数名>(<変数名>: <型>) -> <戻り値の型>:
def gpt(inputs: list[int]) -> list[list[float]]:
  # inputs は [n_seq] の形状を持つ
  # 出力は [n_seq, n_vocab] の形状を持つ

  vocab = ["all", "not", "heroes", "the", "wear", ".", "capes"]
  iputs = [1, 0, 2, 4] # "not" "all" "heroes" "wear"
  output = gpt(inputs) # beep boop neural networkの魔法
  next_token_id = np.aragmax(output[-1]) # next_token_id = 6
  next_token = vocab[next_token_id] # next_token ="capes"
  # 次のトークン予測を行っている
  return output
  ```

### テキストの生成

```python
# トークン予測を繰り返し取得することで、完全な分を生成する。各反復で、予測されたトークンを入力に追加し戻す。
def generate(inputs, n_tokens_to_generate):
  for _ in range(n_tokens_to_generate): # 自己回帰的でコードループ
    outpu = gpt(inputs) # モデルのフォワードパス
    next_id = np.argmax(output[-1]) # 貪欲サンプリング
    inputs.append(int(next_id)) # 予測を入力に追加
  return inputs[len(inputs) - n_tokens_to_generate :] # 生成されたIDのみを返す
  ```
この将来の値を予測し（回帰）、それを入力に追加する（自己）、というプロセスが、GPTを**自己回帰**と表現する理由

### トレーニング

```python
def lm_loss(inputs: list[int], params) -> float:
    # ラベルyは単に入力を1つ左にシフトしたものです。
    #
    # inputs = [not,     all,   heros,   wear,   capes]
    #      x = [not,     all,   heroes,  wear]
    #      y = [all,  heroes,     wear,  capes]
    #
    # もちろん、inputs[-1]に対するラベルはありませんので、xから除外します。
    #
    # そのため、N個の入力に対して、N - 1個の言語モデリング例のペアがあります。
    x, y = inputs[:-1], inputs[1:]
    # x: lower = 0, upper = N-1 の要素まで
    # y: lower = 1, upper = 末尾の要素まで

    # フォワードパス
    # 各位置における予測された次のトークンの確率分布
    output = gpt(x, params)

    # クロスエントロピー損失
    # 全てのN-1例についての平均を取ります。
    loss = np.mean(-np.log(output[y]))

    return loss

def train(texts: list[list[str]], params) -> float:
    for text in texts:
        inputs = tokenizer.encode(text)
        loss = lm_loss(inputs, params)
        gradients = compute_gradients_via_backpropagation(loss, params)
        params = gradient_descent_update_step(gradients, params)
    return params
```

# <font color="blue">セットアップ ここからがpicoGPTの実装</font>

### 必要なパッケージをインスト―ル
- `fire`：Google製のlibraryです。「**コマンドラインからオブジェクトを自由に操作**」できるようになります。  
  詳細については[こちらを参照](https://qiita.com/SaitoTsutomu/items/a5eb827737c9d59af2af)

In [1]:
!pip install fire

- google driveのマウント

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- driveのマウント後であれば左のタブバーから**ファイルのパスのコピー**が可能  
[参考](https://qiita.com/kado_u/items/45b76f9a6f920bf0f786)

In [3]:
 cd /content/drive/MyDrive/データサイエンス/picoGPT/picoGPT

/content/drive/MyDrive/データサイエンス/picoGPT/picoGPT


- 依存関係のインストール

In [29]:
!pip install -r requirements.txt

Ignoring tensorflow-macos: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment


In [30]:
import numpy as np

```python
import numpy as np


def gpt2(inputs, wte, wpe, blocks, ln_f, n_head):
   pass # TODO: これを実装する


def generate(inputs, params, n_head, n_tokens_to_generate):
    from tqdm import tqdm

    for _ in tqdm(range(n_tokens_to_generate), "generating"):  # 自己回帰デコードループ
        logits = gpt2(inputs, **params, n_head=n_head)  # モデルのフォワードパス
        next_id = np.argmax(logits[-1])  # 貪欲サンプリング
        inputs.append(int(next_id))  # 予測を入力に追加

    return inputs[len(inputs) - n_tokens_to_generate :]  # 生成されたidのみを返す


def main(prompt: str, n_tokens_to_generate: int = 40, model_size: str = "124M", models_dir: str = "models"):
    from utils import load_encoder_hparams_and_params

    # 公開されているOpenAI GPT-2ファイルからエンコーダー、hparams、およびparamsを読み込む
    encoder, hparams, params = load_encoder_hparams_and_params(model_size, models_dir)

    # BPEトークナイザーを使用して入力文字列をエンコードする
    input_ids = encoder.encode(prompt)

    # モデルの最大シーケンス長を超えないようにする
    assert len(input_ids) + n_tokens_to_generate < hparams["n_ctx"]

    # 出力idを生成する
    output_ids = generate(input_ids, params, hparams["n_head"], n_tokens_to_generate)

    # idを文字列にデコードする
    output_text = encoder.decode(output_ids)

    return output_text


if __name__ == "__main__":
    import fire

    fire.Fire(main)
```

In [31]:
!pip install utils

In [32]:
from utils import load_encoder_hparams_and_params
encoder, hparams, params = load_encoder_hparams_and_params("124M", "models")

## エンコーダー

`encoder`はGPT-2で使用されるBPEトークナイザー：  
```python
>>> ids = encoder.encode("Not all heroes wear capes.")  
>>> ids  
[3673, 477, 10281, 5806, 1451, 274, 13]
```
```python
>>> encoder.decode(ids)  
"Not all heroes wear capes."  
```
トークナイザーの語彙(encoder.decoder`に保存されてる)を使用して、実際のトークンがどのように見えるかを確認できる：
```python
>>> [encoder.decoder[i] for i in ids]  
['Not', 'Ġall', 'Ġheroes', 'Ġwear', 'Ġcap', 'es', '.']  
```
トークンは時には単語であり（例：Not）、時にはその前にスペースがある単語であることもあります（例：Ġall、Ġ はスペースを表します）、時には単語の一部であることもあります（例：capes は Ġcap と es に分割されます）、そして時には句読点であることもあります（例：.）。  

BPEの良い点の一つは、任意の文字列をエンコードできること。語彙にない何かに出会った場合、それを理解できるサブストリングに分解する：
```python
>>> [encoder.decoder[i] for i in encoder.encode("zjqfl")]  
['z', 'j', 'q', 'fl']  
```
また、語彙のサイズも確認できる：
```python
>>> len(encoder.decoder)  
50257
```

## ハイパーパラメータ

`hparams`は、モデルのハイパーパラメータを含む辞書：
```python
>>> hparams  
{  
"n_vocab": 50257, # 語彙のトークン数  
"n_ctx": 1024, # 入力の最大可能なシーケンス長  
"n_embd": 768, # 埋め込み次元（ネットワークの「幅」を決定する）  
"n_head": 12, # アテンションヘッドの数（n_embdはn_headで割り切れる必要がある）  
"n_layer": 12 # レイヤーの数（ネットワークの「深さ」を決定する）  
}  
```

```python
n_seq = len(inputs) # 入力シーケンスの長さを示す
```

## パラメータ

`params`は、モデルの学習済みの重みを保持するネストされたJSON辞書。JSONの葉ノードはNumPy配列であり、`params`を出力するときに、配列をその形状に置き換える。  
  
この辞書を参照して、GPTを実装する際に重みの形状を確認するために後で戻ってくる必要がある。コード内の変数名は、この辞書のキーと一致させるたに一貫性を持たせる。

```python
import numpy as np

def shape_tree(d):
     if isinstance(d, np.ndarray):
         return list(d.shape)
     elif isinstance(d, list):
         return [shape_tree(v) for v in d]
     elif isinstance(d, dict):
         return {k: shape_tree(v) for k, v in d.items()}
     else:
         ValueError("uh oh")
print(shape_tree(params))
```

```python
import tensorflow as tf
tf_ckpt_path = tf.train.latest_checkpoint("models/124M")
for name, _ in tf.train.list_variables(tf_ckpt_path):
    arr = tf.train.load_variable(tf_ckpt_path, name).squeeze()
    print(f"{name}: {arr.shape}")
```

#  基本レイヤー

GPTアーキテクチャに入る前に、GPTに固有ではない、いくつかの基本的なニューラルネットワークのレイヤーを実装してみる。

## GELU

Gelu-Reluのグラフに注目すると、このグラフには2つの谷がある。そして勾配が0になるのは x = ±1 付近。  
以上から推測すると、この関数は関数の入力を 1 か -1 に近づける勾配成分を持つのではないかと考えられる。（正則化を推進する項が活性化関数に含まれる）  

```python
def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3)))
```


```python
>>> gelu(np.array([[1, 2], [-2, 0.5]]))
array([[ 0.84119,  1.9546 ],
       [-0.0454 ,  0.34571]])
```

In [33]:
def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3)))

## ソフトマックス関数

実数の集合（-∞から∞の間）を確立（0から1の間で、合計が1になる数）に変換するために使用。

```python
>>> x = softmax(np.array([[2, 100], [-5, 0]]))
>>> x
array([[0.00034, 0.99966],
       [0.26894, 0.73106]])
>>> x.sum(axis=-1)
array([1., 1.])
```

In [34]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

## レイヤー正規化

> 名前の通り「レイヤー単位」で，特徴値(グリッド上の各1つの特徴)の正規化を行う．ここでいう「レイヤー(方向での)正規化」とは，RNNで処理している系列中の，
番目の隠れ層(レイヤー)の単位で，各隠れにてデータ正規化を行う事をさす．つまりは，「バッチ内における，レイヤーごとの正規化」がレイヤー正規化である。[出典](https://cvml-expertguide.net/terms/dl/layers/batch-normalization-layer/layer-normalization/#4_%E3%81%BE%E3%81%A8%E3%82%81)



In [35]:
def layer_norm(x, g, b, eps: float = 1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    variance = np.var(x, axis=-1, keepdims=True)
    x = (x - mean) / np.sqrt(variance + eps)  # 最後の軸において平均=0、分散=1になるようにxを正規化
    return g * x + b  # gamma/betaパラメータでスケールとオフセットを行う

```python
def layer_norm(x, g, b, eps: float = 1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    variance = np.var(x, axis=-1, keepdims=True)
    x = (x - mean) / np.sqrt(variance + eps)  # 最後の軸において平均=0、分散=1になるようにxを正規化
    return g * x + b  # gamma/betaパラメータでスケールとオフセットを行う
```

## 線形

標準的な行列乗算 + バイアス：
```python
def linear(x, w, b):  # [m, in], [in, out], [out] -> [m, out]
    return x @ w + b
```

In [36]:
def linear(x, w, b):  # [m, in], [in, out], [out] -> [m, out]
    return x @ w + b

# GPTアーキテクチャ

GPTアーキテクチャは、Transformerに従う：

**Transformerについて**
  
> この文章に含まれる単語のように、連続したデータの関係を追跡することにより、文章ひいては意味を学習するニューラルネットワーク。[出典](https://blogs.nvidia.co.jp/2022/04/13/what-is-a-transformer-model/)


高レベルで、GPTアーキテクチャは3つのセクションを持っています：
 - テキスト+位置**エンベッディング**
 - Transformer **デコーダスタック**
 - **語彙への投影** ステップ
 コードでは、以下のようになる：
```python
def gpt2(inputs, wte, wpe, blocks, ln_f, n_head):  # [n_seq] -> [n_seq, n_vocab]
    # トークンと位置埋め込みの追加
    x = wte[inputs] + wpe[range(len(inputs))]  # [n_seq] -> [n_seq, n_embd]

    # n_layer Transformerブロックを通じてのフォワードパス
    for block in blocks:
        x = transformer_block(x, **block, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # 語彙への射影
    x = layer_norm(x, **ln_f)  # [n_seq, n_embd] -> [n_seq, n_embd]
    return x @ wte.T  # [n_seq, n_embd] -> [n_seq, n_vocab]
```

In [37]:
def gpt2(inputs, wte, wpe, blocks, ln_f, n_head):  # [n_seq] -> [n_seq, n_vocab]
    # トークンと位置埋め込みの追加
    x = wte[inputs] + wpe[range(len(inputs))]  # [n_seq] -> [n_seq, n_embd]

    # n_layer Transformerブロックを通じてのフォワードパス
    for block in blocks:
        x = transformer_block(x, **block, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # 語彙への射影
    x = layer_norm(x, **ln_f)  # [n_seq, n_embd] -> [n_seq, n_embd]
    return x @ wte.T  # [n_seq, n_embd] -> [n_seq, n_vocab]

## 埋め込み


#### トークンの埋め込み

トークンIDの相対的な大きさが誤った情報を伝える恐れがある。また、単一の数値は、ニューラルネットワークが扱うには次元性が十分ではない。そのため、単語ベクトルを利用するが、特に学習された埋め込み行を介す。  
**つまり、トレーニングの開始時にランダムに初期化されr、その後勾配降下法を通じて更新される**

```python
wte[inputs] # [n_seq] -> [n_seq, n_embd]
```

## 位置埋め込み

Transformerアーキテクチャの特徴の一つは一を考慮しないこと。  
**入力をランダムにシャッフルし、出力を適切にアンシャッフルした場合、入力を一切シャッフルしなかった場合と同じ**  


```python
wpe[range(len(inputs))] # [n_seq] -> [n_seq, n_embd]
```

## 組み合わせ

トークンの一の埋め込みを加算することで、トークンと位置情報の両方をエンコードする組み合わせを得ることができる。

```python
# トークン + 位置埋め込み
x = wte[inputs] + wpe[range(len(inputs))]  # [n_seq] -> [n_seq, n_embd]

# x[i] は i 番目の単語の単語埋め込みと i 番目の位置の位置埋め込みを表します
```

## デコーダスタック

ここでディープラーニングに入ってくる。埋め込みを`n_layer`のTransformerデコーダブロックのスタックを通して渡す。  
より多くの層を積み重ねることで、ネットワークがどれだけ「深い」かを制御できる。また、`n_embd`値はネットワークがどれだけ「幅広いか」を制御することができる。

```python
# n_layer Transformerブロックを通じた順伝播
for block in blocks:
    x = transformer_block(x, **block, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]
```

## 語彙への投影

ここで最後のTransformerブロックの出力を5以上の確率分布へ投影する：
```python
# 語彙への投影
x = layer_norm(x, **ln_f)  # [n_seq, n_embd] -> [n_seq, n_embd]
return x @ wte.T  # [n_seq, n_embd] -> [n_seq, n_vocab]
```
  
GPTが事前トレーニングされた後、言語モデリングヘッドを何らかの分類タスクのための**分類ヘッド**など、他の種類の投影に交換することが出来る。  


## デコーダブロック

Transformerデコーダブロックは、2つのサブレイヤーで構成される：  
1. マルチヘッド因果的自己注意
2. 位置ごとのフィードフォワードニューラルネットワーク  
※各サブレイヤーは入力にレイヤー正規化を利用し残差接続（サブレイヤーの入力をサブレイヤーの出力をサーブレイヤーの出力に加算）を使用：
  
```python
def transformer_block(x, mlp, attn, ln_1, ln_2, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # マルチヘッド因果的自己注意
    x = x + mha(layer_norm(x, **ln_1), **attn, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # 位置ごとのフィードフォワードネットワーク
    x = x + ffn(layer_norm(x, **ln_2), **mlp)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x
```

In [38]:
def transformer_block(x, mlp, attn, ln_1, ln_2, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # マルチヘッド因果的自己注意
    x = x + mha(layer_norm(x, **ln_1), **attn, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # 位置ごとのフィードフォワードネットワーク
    x = x + ffn(layer_norm(x, **ln_2), **mlp)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x

**勾配消失問題**とは  
    
ニューラルネットワークが深くなるほど勾配が小さくなり、重みの更新が十分にされなり学習が停滞してしまう現象のこと。主な原因は、活性化関数と重みの初期値にある。  
※勾配とは、損失関数をニューラルネットワークのパラメータ（重み）で微分したもの。ニューラルネットワークは重みが何度も更新されることによって学習が進み完成度を上げる仕組みだが、勾配が小さくなる（ゼロ近づく）と方程式の第2項以降の微分がほぼゼロになり、重みが更新されなくなる。

## 位置ごとのフィードフォワードネットワーク

**フィードフォワードネットワーク（feed forward network）**
  
入力層から出力層へと一方項に情報が流れるネットワーク構造を持つ。  
このネットワークの効果は、中間層における多数のニューロンを通じてデータから特徴を抽出し、それらを組み合わせることにより、より高度なパターンや関係性を学習することである。  
[参考：フィードフォワードネットワーク：ディープラーニングの基礎](https://reinforz.co.jp/bizmedia/24900/)  
  
単なる2層のマルチレイヤ―パーセプトロン：
```python
def ffn(x, c_fc, c_proj):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # アップへのプロジェクト
    a = gelu(linear(x, **c_fc))  # [n_seq, n_embd] -> [n_seq, 4*n_embd]

    # ダウンへのプロジェクト
    x = linear(a, **c_proj)  # [n_seq, 4*n_embd] -> [n_seq, n_embd]

    return x
```


In [39]:
def ffn(x, c_fc, c_proj):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # アップへのプロジェクト
    a = gelu(linear(x, **c_fc))  # [n_seq, n_embd] -> [n_seq, 4*n_embd]

    # ダウンへのプロジェクト
    x = linear(a, **c_proj)  # [n_seq, 4*n_embd] -> [n_seq, n_embd]

    return x

## マルチヘッド因果自己注意

各単語に分解して説明することで、この概念を理解しよう：
  1. 注意（Attention）
  2. 自己（Self）
  3. 因果（Causal）
  4. マルチヘッド（Multi-Head）
  
※トークナイザーの語彙(encoder.decoder`に保存されてる)を使用して、実際のトークンがどのように見えるかを確認してるみたいだー
```python
>>> [encoder.decoder[i] for i in ids]  
['注意', '自己', '因果', 'マルチヘッド']  
```

#### 注意（Attention）

**Scaled Dot-Product Attention**  
内積を利用したベクトル間の類似性に元ずく返還を行う。入力はQuery、Key、valueの3つ。Queryに基づいてKeyに何らかの変更を施し、valueを取り出す操作をする。  
QueryベクトルとKeyの類似性に基づいてValueの各ベクトルの線形結合を計算する。  
例えば、KeyとValueには「I have a pen.」を表す行列を、Queryには「have」を表すベクトルを与えると、「have」のベクトルとの類似性に基づいて文章全体の単語ベクトルを用いて線形結合を計算しベクトルを出力する。出力されるベクトルのサイズはQueryのベクトルと同じ。その為、出力ベクトルを単語ベクトルとして考えることができる。これを再帰的に行えば、文章を生成することが可能である。  
![](https://developers.agirobots.com/jp/wp-content/uploads/2023/02/image-29-1024x578.png)


```python
def attention(q, k, v):  # [n_q, d_k], [n_k, d_k], [n_k, d_v] -> [n_q, d_v]
    return softmax(q @ k.T / np.sqrt(q.shape[-1])) @ v
```

In [40]:
def attention(q, k, v):  # [n_q, d_k], [n_k, d_k], [n_k, d_v] -> [n_q, d_v]
    return softmax(q @ k.T / np.sqrt(q.shape[-1])) @ v

#### 自己（Self）

`Query`、`Key`、`Value`がすべて同じソースから来る場合、自己注意を実行：
```python
def self_attention(x): # [n_seq, n_embd] -> [n_seq, n_embd]
    return attention(q=x, k=x, v=x)
```
  
`Query`、`Key`、`Value`および注意出力のための投影を導入：
```python
def self_attention(x, w_k, w_q, w_v, w_proj): # [n_seq, n_embd] -> [n_seq, n_embd]
    # qkvの投影
    q = x @ w_k # [n_seq, n_embd] @ [n_embd, n_embd] -> [n_seq, n_embd]
    k = x @ w_q # [n_seq, n_embd] @ [n_embd, n_embd] -> [n_seq, n_embd]
    v = x @ w_v # [n_seq, n_embd] @ [n_embd, n_embd] -> [n_seq, n_embd]

    # 自己注意の実行
    x = attention(q, k, v) # [n_seq, n_embd] -> [n_seq, n_embd]

    # 出力の投影
    x = x @ w_proj # [n_seq, n_embd] @ [n_embd, n_embd] -> [n_seq, n_embd]

    return x
```
  
`w_q`、`w_k`、`w_v@を単一の行列w_fcに組み合わせ、投影を実行し、その結果を分割することで、行列乗算の数を 4 から 2 に減らす：
```python
def self_attention(x, w_fc, w_proj): # [n_seq, n_embd] -> [n_seq, n_embd]
    # qkvの投影
    x = x @ w_fc # [n_seq, n_embd] @ [n_embd, 3*n_embd] -> [n_seq, 3*n_embd]

    # qkvに分割
    q, k, v = np.split(x, 3, axis=-1) # [n_seq, 3*n_embd] -> 3 of [n_seq, n_embd]

    # 自己注意の実行
    x = attention(q, k, v) # [n_seq, n_embd] -> [n_seq, n_embd]

    # 出力の投影
    x = x @ w_proj # [n_seq, n_embd] @ [n_embd, n_embd] = [n_seq, n_embd]

    return x
```
  
現代のGPUが連続して発生する3つの小さな行列演算よりも1つの大きな行列乗算をより有効に利用で機少し効率的。
  
GPT-2 の実装に合わせてバイアスベクトルを追加し、linear関数を使用し、params辞書に合わせてパラメータの名前を変更：
```python
def self_attention(x, c_attn, c_proj): # [n_seq, n_embd] -> [n_seq, n_embd]
    # qkvの投影
    x = linear(x, **c_attn) # [n_seq, n_embd] -> [n_seq, 3*n_embd]

    # qkvに分割
    q, k, v = np.split(x, 3, axis=-1) # [n_seq, 3*n_embd] -> 3 of [n_seq, n_embd]

    # 自己注意の実行
    x = attention(q, k, v) # [n_seq, n_embd] -> [n_seq, n_embd]

    # 出力の投影
    x = linear(x, **c_proj) # [n_seq, n_embd] @ [n_embd, n_embd] = [n_seq, n_embd]

    return x
```



#### 因果（Causl）

どのような入力を受け取りモデルが学習し出力するか学習時にすでに知っているような状態になってしまわないようにするために、**入力が未来を見ることが出来ないように隠すかマスクする必要がある**。
一般に、入力のすべてのクエリが未来を見ることを防ぐ為に、*j > i*であるすべての位置*i*, *j*を`0`に設定する：(これを**マスキング**と呼ぶ)

```python
       not    all    heroes wear   capes
   not 0.116  0.     0.     0.     0.
   all 0.180  0.397  0.     0.     0.
heroes 0.156  0.453  0.028  0.     0.
  wear 0.499  0.055  0.133  0.017  0.
 capes 0.089  0.290  0.240  0.228  0.153
 ```
   
しかし、これは`softmax`が適用後であり、行列の合計が1にならないため、`softmax`が適用される前に注意行列を修正する必要がある。マスクされるエントリを`softmax`の前に$-∞$に設定することで達成可能：

```python  
def attention(q, k, v, mask):   # [n_q, d_k], [n_k, d_k], [n_k, d_v], [n_q, n_k] -> [n_q, d_v]
  return softmax(q @ k.T / np.sqrt(q.shape[-1]) + mask) @ v
```
  
これらを、すべてまとめると：
```python
def attention(q, k, v, mask):  # q, k, v はそれぞれクエリ、キー、バリューを表し、mask は注意を適用する範囲を制御します。戻り値は注意後の出力です。
    return softmax(q @ k.T / np.sqrt(q.shape[-1]) + mask) @ v  # スケール済みドット積注意を計算し、適用します。

def causal_self_attention(x, c_attn, c_proj):  # x は入力シーケンス、c_attn と c_proj はそれぞれ注意と出力投影のための設定を含む。
    # qkv 投影
    x = linear(x, **c_attn)  # 入力を線形変換して、q, k, v のための3倍の次元を持つベクトルにします。

    # qkv に分割
    q, k, v = np.split(x, 3, axis=-1)  # 上記のベクトルを q, k, v に分割します。

    # 未来の入力を隠すための因果マスク
    causal_mask = (1 - np.tri(x.shape[0]), dtype=x.dtype) * -1e10  # 自己注意で将来の情報が現れないようにするマスクを作成します。

    # 因果的自己注意を実行
    x = attention(q, k, v, causal_mask)  # 因果的自己注意を適用します。

    # 出力投影
    x = linear(x, **c_proj)  # 注意後の出力を再び線形変換して最終的な出力を得ます。

    return x
```

In [41]:
def attention(q, k, v, mask):  # q, k, v はそれぞれクエリ、キー、バリューを表し、mask は注意を適用する範囲を制御します。戻り値は注意後の出力です。
    return softmax(q @ k.T / np.sqrt(q.shape[-1]) + mask) @ v  # スケール済みドット積注意を計算し、適用します。

#### マルチヘッド

Query、Key、Valueを**ヘッド**に分割することで、実装をさらに改善可能：
```python
def mha(x, c_attn, c_proj, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # qkvの投影
    x = linear(x, **c_attn)  # [n_seq, n_embd] -> [n_seq, 3*n_embd]

    # qkvに分割
    qkv = np.split(x, 3, axis=-1)  # [n_seq, 3*n_embd] -> [3, n_seq, n_embd]

    # ヘッドに分割
    qkv_heads = list(map(lambda x: np.split(x, n_head, axis=-1), qkv))  # [3, n_seq, n_embd] -> [3, n_head, n_seq, n_embd/n_head]

    # 未来の入力を見ることができないように因果マスクを適用
    causal_mask = (1 - np.tri(x.shape[0]), dtype=x.dtype) * -1e10  # [n_seq, n_seq]

    # 各ヘッドで注意を実行
    out_heads = [attention(q, k, v, causal_mask) for q, k, v in zip(*qkv_heads)]  # [3, n_head, n_seq, n_embd/n_head] -> [n_head, n_seq, n_embd/n_head]

    # ヘッドを結合
    x = np.hstack(out_heads)  # [n_head, n_seq, n_embd/n_head] -> [n_seq, n_embd]

    # 出力の投影
    x = linear(x, **c_proj)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x
```
  
1. `Query`、`Kev`、`Value`を分割
2. 各ヘッドに対して注意を計算
3. 出力の結合
  
これにより、次元が減少するが、モデルはAttentionを通じて関係をモデリングする際に、追加の*部分空間*を利用できる。


In [42]:
def mha(x, c_attn, c_proj, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # qkvの投影
    x = linear(x, **c_attn)  # [n_seq, n_embd] -> [n_seq, 3*n_embd]

    # qkvに分割
    qkv = np.split(x, 3, axis=-1)  # [n_seq, 3*n_embd] -> [3, n_seq, n_embd]

    # ヘッドに分割
    qkv_heads = list(map(lambda x: np.split(x, n_head, axis=-1), qkv))  # [3, n_seq, n_embd] -> [3, n_head, n_seq, n_embd/n_head]

    # 未来の入力を見ることができないように因果マスクを適用
    # causal_mask = (1 - np.tri(x.shape[0]), dtype=x.dtype) * -1e10  # [n_seq, n_seq]
    causal_mask = (1 - np.tri(x.shape[0], dtype=x.dtype)) * -1e10  # [n_seq, n_seq]

    # 各ヘッドで注意を実行
    out_heads = [attention(q, k, v, causal_mask) for q, k, v in zip(*qkv_heads)]  # [3, n_head, n_seq, n_embd/n_head] -> [n_head, n_seq, n_embd/n_head]

    # ヘッドを結合
    x = np.hstack(out_heads)  # [n_head, n_seq, n_embd/n_head] -> [n_seq, n_embd]

    # 出力の投影
    x = linear(x, **c_proj)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x

In [43]:
def generate(inputs, params, n_head, n_tokens_to_generate):
    from tqdm import tqdm

    for _ in tqdm(range(n_tokens_to_generate), "generating"):  # 自己回帰デコードループ
        logits = gpt2(inputs, **params, n_head=n_head)  # モデルのフォワードパス
        next_id = np.argmax(logits[-1])  # 貪欲サンプリング
        inputs.append(int(next_id))  # 予測を入力に追加

    return inputs[len(inputs) - n_tokens_to_generate :]  # 生成されたidのみを返す

In [48]:
def main(prompt: str, n_tokens_to_generate: int = 40, model_size: str = "124M", models_dir: str = "models"):
    from utils import load_encoder_hparams_and_params

    # 公開されているOpenAI GPT-2ファイルからエンコーダー、hparams、およびparamsを読み込む
    encoder, hparams, params = load_encoder_hparams_and_params(model_size, models_dir)

    # BPEトークナイザーを使用して入力文字列をエンコードする
    input_ids = encoder.encode(prompt)

    # モデルの最大シーケンス長を超えないようにする
    assert len(input_ids) + n_tokens_to_generate < hparams["n_ctx"]

    # 出力idを生成する
    output_ids = generate(input_ids, params, hparams["n_head"], n_tokens_to_generate)

    # idを文字列にデコードする
    output_text = encoder.decode(output_ids)

    return output_text

# 最後にすべてをまとめて実行

ここでは例として与える文字列を  
"Alan Turing theorized that computers would one day become" としている。  
ここで、出力は  
the most powerful machines on the planet.  
が返ってくれば成功！

In [52]:
def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3)))


def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)


def layer_norm(x, g, b, eps: float = 1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    variance = np.var(x, axis=-1, keepdims=True)
    x = (x - mean) / np.sqrt(variance + eps)  # 最後の軸において平均=0、分散=1になるようにxを正規化
    return g * x + b  # gamma/betaパラメータでスケールとオフセットを行う


def linear(x, w, b):  # [m, in], [in, out], [out] -> [m, out]
    return x @ w + b


def ffn(x, c_fc, c_proj):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # アップへのプロジェクト
    a = gelu(linear(x, **c_fc))  # [n_seq, n_embd] -> [n_seq, 4*n_embd]

    # ダウンへのプロジェクト
    x = linear(a, **c_proj)  # [n_seq, 4*n_embd] -> [n_seq, n_embd]

    return x


def attention(q, k, v, mask):  # q, k, v はそれぞれクエリ、キー、バリューを表し、mask は注意を適用する範囲を制御します。戻り値は注意後の出力です。
    return softmax(q @ k.T / np.sqrt(q.shape[-1]) + mask) @ v  # スケール済みドット積注意を計算し、適用します。


def mha(x, c_attn, c_proj, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # qkvの投影
    x = linear(x, **c_attn)  # [n_seq, n_embd] -> [n_seq, 3*n_embd]

    # qkvに分割
    qkv = np.split(x, 3, axis=-1)  # [n_seq, 3*n_embd] -> [3, n_seq, n_embd]

    # ヘッドに分割
    qkv_heads = list(map(lambda x: np.split(x, n_head, axis=-1), qkv))  # [3, n_seq, n_embd] -> [3, n_head, n_seq, n_embd/n_head]

    # 未来の入力を見ることができないように因果マスクを適用
    # causal_mask = (1 - np.tri(x.shape[0]), dtype=x.dtype) * -1e10  # [n_seq, n_seq]
    causal_mask = (1 - np.tri(x.shape[0], dtype=x.dtype)) * -1e10  # [n_seq, n_seq]

    # 各ヘッドで注意を実行
    out_heads = [attention(q, k, v, causal_mask) for q, k, v in zip(*qkv_heads)]  # [3, n_head, n_seq, n_embd/n_head] -> [n_head, n_seq, n_embd/n_head]

    # ヘッドを結合
    x = np.hstack(out_heads)  # [n_head, n_seq, n_embd/n_head] -> [n_seq, n_embd]

    # 出力の投影
    x = linear(x, **c_proj)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x


def transformer_block(x, mlp, attn, ln_1, ln_2, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # マルチヘッド因果的自己注意
    x = x + mha(layer_norm(x, **ln_1), **attn, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # 位置ごとのフィードフォワードネットワーク
    x = x + ffn(layer_norm(x, **ln_2), **mlp)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x


def gpt2(inputs, wte, wpe, blocks, ln_f, n_head):  # [n_seq] -> [n_seq, n_vocab]
    # トークンと位置埋め込みの追加
    x = wte[inputs] + wpe[range(len(inputs))]  # [n_seq] -> [n_seq, n_embd]

    # n_layer Transformerブロックを通じてのフォワードパス
    for block in blocks:
        x = transformer_block(x, **block, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # 語彙への射影
    x = layer_norm(x, **ln_f)  # [n_seq, n_embd] -> [n_seq, n_embd]
    return x @ wte.T  # [n_seq, n_embd] -> [n_seq, n_vocab]


def generate(inputs, params, n_head, n_tokens_to_generate):
    from tqdm import tqdm

    for _ in tqdm(range(n_tokens_to_generate), "generating"):  # 自己回帰デコードループ
        logits = gpt2(inputs, **params, n_head=n_head)  # モデルのフォワードパス
        next_id = np.argmax(logits[-1])  # 貪欲サンプリング
        inputs.append(int(next_id))  # 予測を入力に追加

    return inputs[len(inputs) - n_tokens_to_generate :]  # 生成されたidのみを返す


def main(prompt: str, n_tokens_to_generate: int = 40, model_size: str = "124M", models_dir: str = "models"):
    from utils import load_encoder_hparams_and_params

    # 公開されているOpenAI GPT-2ファイルからエンコーダー、hparams、およびparamsを読み込む
    encoder, hparams, params = load_encoder_hparams_and_params(model_size, models_dir)

    # BPEトークナイザーを使用して入力文字列をエンコードする
    input_ids = encoder.encode(prompt)

    # モデルの最大シーケンス長を超えないようにする
    assert len(input_ids) + n_tokens_to_generate < hparams["n_ctx"]

    # 出力idを生成する
    output_ids = generate(input_ids, params, hparams["n_head"], n_tokens_to_generate)

    # idを文字列にデコードする
    output_text = encoder.decode(output_ids)

    return output_text

main("Alan Turing theorized that computers would one day become")

generating: 100%|██████████| 40/40 [00:16<00:00,  2.38it/s]


' the most powerful machines on the planet.\n\nThe computer is a machine that can perform complex calculations, and it can perform these calculations in a way that is very similar to the human brain.\n'

- Pythonスクリプトとして起動された場合のみ配下の処理が実行  
```python
if __name__ = "__main___":
  import fire
  fire.Fire(main)
```

# その他 memo



####  基本的な変数の型[イミュータブル]

```
<変数名>:<型>
<変数名>:<型> = <初期値>
```

#### 関数の型  

基本形

```
def <関数名>(<関数名>: <型>) -> <戻り値の型>:
	...
```
実査に使う場合以下のようになる
```python
def add(x: int, y: int) -> int:
	return x + y
```
戻り値がない関数の場合は、戻り値の型をNoneとして設定する。
```python
def log_print(number: int) -> None:
	print('値は{}です'.format(number))

b = log_pirnt(12) # mypy error
```

#### 配列の最大要素のインデックスを返すNumPyのargmax関数の使い方

[参考サイト](https://deepage.net/features/numpy-argmax.html)
NumPyのargmax関数は、**多次元配列の中の最大値の要素を持つインデックスを返す関数**。`np.max`を使うと、最大値の要素を返すことができる。`argmax`は、最大の要素のインデックスを返す。
```python
np.argmax(a, axis = None, out = None)
```
使い方
`np.argmax`は、第一引数に最大値を取得したい配列を指定。また、最後の引数の`out`はあまり使用しない。出力の配列を予め作っておいた配列にしたい場合は指定する。
※最小のインデックスを取得したい場合は `argmin` を使うことで取得可能
- 次元が増えると考えるのが少し難しくなる

#### インデックスのスライスの基本

[参考サイト](https://atmarkit.itmedia.co.jp/ait/articles/2012/08/news013.html)
```python
mylist = list[range(10)]
print(mylist) # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# インデックスによるアクセス  
n = mylist[1]  # 正数は先頭からのインデックス。先頭の要素のインデックスが0  
print(n)  # 1  
n = mylist[-2]  # 負数は末尾からのインデックス。末尾の要素のインデックスが-1
print(n)  # 8  
  
# スライスによるアクセス  
mylist = list(range(10))  
  
s = mylist[0:5]  # 0～4番目の5要素を取り出す  
print(s)  # [0, 1, 2, 3, 4]
```
pythonでは、ほかの言語における配列はリストとして実装されているため、その要素にはインデックスやスライスといった機構を通じてアクセスできる。規範は以下
- インデックス：かっこ`[]`内にアクセスしたい要素のインデックスを指定
- スライス：かっこ`[]`内にアクセスしたい要素の範囲を「**lower_bound:upper_bound:stride**」のかたちで記述
  lower_boundはリストの中でスライスの始まる位置
  upper_boundはスライスの終わる位置
  strideは増分（いずれも省略可）